<a href="https://colab.research.google.com/github/konamilk/mlb-player-digital-engagement-forecasting/blob/main/MLB_baseline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from requests import get
NOTEBOOK = get('http://172.28.0.2:9000/api/sessions').json()[0]['name'].split('.')[0]
NOTEBOOK

'MLB_baseline'

In [2]:
import sys
from pathlib import Path
if 'google.colab' in sys.modules:
    INPUT = Path('/content/input/')
elif 'kaggle_web_client' in sys.modules:
    INPUT = Path('../input/')

In [3]:
if 'google.colab' in sys.modules:
  !mkdir ~/.kaggle
  !cp /content/drive/MyDrive/.kaggle/kaggle.json ~/.kaggle
  !chmod 600 ~/.kaggle/kaggle.json
  !pip install kaggle
  !pip install git+https://github.com/pfnet-research/xfeat.git

mkdir: cannot create directory ‘/root/.kaggle’: File exists
  Cloning https://github.com/pfnet-research/xfeat.git to /tmp/pip-req-build-aos_hz2m
  Running command git clone -q https://github.com/pfnet-research/xfeat.git /tmp/pip-req-build-aos_hz2m
  Created wheel for xfeat: filename=xfeat-0.1.1-cp37-none-any.whl size=39635 sha256=ca1afee1c6c136e2fbf14b957a48f4398afec259619fd58a15bcb533d047c04a
  Stored in directory: /tmp/pip-ephem-wheel-cache-v1irezr1/wheels/82/f3/da/060c6ceac1125aa285b041284b7ec5324be345b865c713da9c
Successfully built xfeat


In [4]:
if 'google.colab' in sys.modules:
  !kaggle competitions download -c mlb-player-digital-engagement-forecasting
  !mkdir input
  !unzip -o '*.zip' -d ./input/
  !rm *.zip
  !mv *.csv ./input/

competition.cpython-37m-x86_64-linux-gnu.so: Skipping, found more recently modified local copy (use --force to force download)
__init__.py: Skipping, found more recently modified local copy (use --force to force download)
  0% 0.00/3.96M [00:00<?, ?B/s]
100% 3.96M/3.96M [00:00<00:00, 59.9MB/s]
  0% 0.00/173k [00:00<?, ?B/s]
100% 173k/173k [00:00<00:00, 24.0MB/s]
 97% 456M/470M [00:03<00:00, 133MB/s]
100% 470M/470M [00:03<00:00, 130MB/s]
  0% 0.00/824 [00:00<?, ?B/s]
100% 824/824 [00:00<00:00, 794kB/s]
  0% 0.00/820k [00:00<?, ?B/s]
100% 820k/820k [00:00<00:00, 115MB/s]
  0% 0.00/3.68k [00:00<?, ?B/s]
100% 3.68k/3.68k [00:00<00:00, 3.70MB/s]
  0% 0.00/191k [00:00<?, ?B/s]
100% 191k/191k [00:00<00:00, 62.5MB/s]
mkdir: cannot create directory ‘input’: File exists
Archive:  train.csv.zip
  inflating: ./input/train.csv       

Archive:  example_test.csv.zip
  inflating: ./input/example_test.csv  

2 archives were successfully processed.


In [5]:
import numpy as np
import pandas as pd
import gc
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
import torch
import torch.nn as nn
import torch.functional as F
from torch.utils import data

In [6]:
TRAIN = INPUT/'train.csv'
TEAMS = INPUT/'awards.csv'
PLAYERS = INPUT/'players.csv'
AWARDS = INPUT/'awards.csv'
SEASONS = INPUT/'seasons.csv'

In [7]:
df = pd.read_csv(TRAIN)

In [8]:
def unpack_json(json_str):
    return np.nan if pd.isna(json_str) else pd.read_json(json_str)

In [9]:
import datetime
def next_date_as_int(date_as_int: int):
  dd = date_as_int % 100
  date_as_int = date_as_int // 100
  mm = date_as_int % 100
  yyyy = date_as_int // 100
  dt = datetime.datetime(yyyy, mm, dd)
  next_dt = dt + datetime.timedelta(days=1)
  return next_dt.year * 10000 + next_dt.month * 100 + next_dt.day

In [10]:
class TrainDataset(object):
  def __init__(self,df_train, batch_size=5):
    self.df_train = df_train
    self.current = 0
    self.batch_size = batch_size
    self.df_example_sample_submission = pd.read_csv(INPUT/'example_sample_submission.csv')
    self.playerId =  self.df_example_sample_submission[self.df_example_sample_submission.date == self.df_example_sample_submission.date[0]].date_playerId.map(lambda x: int(x.split('_')[1]))
    self.playerId.name = 'playerId'
  
  def __iter__(self):
    return self

  def __next__(self):
    start = self.current * self.batch_size
    end = (self.current + 1) * self.batch_size
    self.current += 1

    if start >= self.df_train.shape[0]:
      raise StopIteration()
    
    dates = self.df_train[start:end].date.unique()

    temps = []
    for date in dates:
      df_temp = pd.DataFrame(self.playerId)
      df_temp['date'] = date
      df_temp['date_playerId'] = str(next_date_as_int(date)) + '_' + df_temp['playerId'].astype(str)
      df_temp['target1'] = 0.0
      df_temp['target2'] = 0.0
      df_temp['target3'] = 0.0
      df_temp['target4'] = 0.0
      df_temp = df_temp.drop(['playerId'], axis=1)
      temps.append(df_temp)

    return  self.df_train[start:end].drop('nextDayPlayerEngagement', axis=1), pd.concat(temps, axis=0), self.df_train[start:end].nextDayPlayerEngagement

In [114]:
from xfeat import SelectCategorical, LabelEncoder, Pipeline, ConcatCombination, SelectNumerical, \
    ArithmeticCombinations, TargetEncoder, aggregation, GBDTFeatureSelector, GBDTFeatureExplorer

class PreProcessor(object):
  def __init__(self):
    self.label_encoder = Pipeline([
      SelectCategorical(exclude_cols=['engagementMetricsDate']),
      LabelEncoder(output_suffix=""),
      ])

  def pre_process(self, df_dairy: pd.DataFrame, df_submission: pd.DataFrame, phase = "train") -> pd.DataFrame:

    X = df_submission.drop(['target1', 'target2', 'target3', 'target4'], axis=1)
    X['date'] = pd.to_datetime(X['date'].astype(str))
    X['year'] = X['date'].dt.year
    X['month'] = X['date'].dt.month
    X['day'] = X['date'].dt.day
    X['dayofweek'] = X['date'].dt.dayofweek
    X['engagementMetricsDate'] = pd.to_datetime(X['date_playerId'].str.split('_', expand = True)[0])
    X['playerId'] = X['date_playerId'].str.split('_', expand = True)[1].astype(int)

    # join rosters
    df_rosters = pd.concat(map(lambda x: unpack_json(x), df_dairy[df_dairy['rosters'].notnull()]['rosters']), axis=0)
    df_rosters['gameDate'] = pd.to_datetime(df_rosters['gameDate'])
    X = pd.merge(X, df_rosters, how='left', left_on=['date', 'playerId'], right_on=['gameDate', 'playerId']).drop(['gameDate','status'], axis=1)
    del df_rosters

    # join player twitter followers
    df_playerTwitterFollowers = pd.concat(map(lambda x: unpack_json(x),df_dairy[df_dairy['playerTwitterFollowers'].notnull()]['playerTwitterFollowers']),axis=0)
    df_playerTwitterFollowers = df_playerTwitterFollowers[['date', 'playerId', 'numberOfFollowers']]
    df_playerTwitterFollowers['year'] = df_playerTwitterFollowers['date'].dt.year
    df_playerTwitterFollowers['month'] = df_playerTwitterFollowers['date'].dt.month
    df_playerTwitterFollowers = df_playerTwitterFollowers.drop('date', axis=1)
    X = pd.merge(X, df_playerTwitterFollowers, how='left', on=['playerId', 'year', 'month'])
    del df_playerTwitterFollowers

    # join team twitter followers
    df_teamTwitterFollowers = pd.concat(map(lambda x: unpack_json(x),df_dairy[df_dairy['teamTwitterFollowers'].notnull()]['teamTwitterFollowers']),axis=0)
    df_teamTwitterFollowers = df_teamTwitterFollowers[['date', 'teamId', 'numberOfFollowers']]
    df_teamTwitterFollowers['year'] = df_teamTwitterFollowers['date'].dt.year
    df_teamTwitterFollowers['month'] = df_teamTwitterFollowers['date'].dt.month
    df_teamTwitterFollowers = df_teamTwitterFollowers.drop('date', axis=1)
    X = pd.merge(X, df_teamTwitterFollowers, how='left', on=['teamId', 'year', 'month'])
    del df_teamTwitterFollowers

    # join games
    df_games = pd.concat(map(lambda x: unpack_json(x),df_dairy[df_dairy['games'].notnull()]['games']),axis=0)
    new_columns = ['gamePk', 'gameType', 'season', 'gameDate', 'gameTimeUTC', 'resumeDate',
          'resumedFrom', 'codedGameState', 'detailedGameState', 'isTie',
          'gameNumber', 'doubleHeader', 'dayNight', 'scheduledInnings',
          'gamesInSeries', 'seriesDescription', 'teamId', 'teamName',
          'teamAbbrev', 'teamWins', 'teamLosses', 'teamWinPct', 'teamWinner',
          'teamScore', 'opponentId', 'opponentName', 'opponentAbbrev', 'opponentWins',
          'opponentLosses', 'opponentWinPct', 'opponentWinner', 'opponentScore']
    df_games_home = df_games.copy()
    df_games_home.columns = new_columns
    df_games_away = df_games.copy()[['gamePk', 'gameType', 'season', 'gameDate', 'gameTimeUTC', 'resumeDate',
          'resumedFrom', 'codedGameState', 'detailedGameState', 'isTie',
          'gameNumber', 'doubleHeader', 'dayNight', 'scheduledInnings',
          'gamesInSeries', 'seriesDescription', 'awayId', 'awayName', 'awayAbbrev', 'awayWins',
          'awayLosses', 'awayWinPct', 'awayWinner', 'awayScore', 'homeId', 'homeName',
          'homeAbbrev', 'homeWins', 'homeLosses', 'homeWinPct', 'homeWinner',
          'homeScore']]
    df_games_away.columns = new_columns
    df_games_2 = pd.concat([df_games_home, df_games_away], axis=0).drop(['teamName', 'teamAbbrev', 'opponentName', 'opponentAbbrev', 'opponentWinner'], axis=1)
    df_games_2['date'] = pd.to_datetime(df_games_2['gameDate'])
    X = pd.merge(X, df_games_2, how='left', on=['teamId', 'date'])
    del new_columns, df_games, df_games_home, df_games_away, df_games_2


    # label encoding
    if phase == 'train':
      X = X.drop(['date', 'date_playerId'], axis=1)
      encoded_X = self.label_encoder.fit_transform(X)
      X = pd.concat([X['engagementMetricsDate'], encoded_X, SelectNumerical().fit_transform(X)], axis=1)
    elif phase == 'eval':
      X = X.drop(['date', 'date_playerId'], axis=1)
      encoded_X = self.label_encoder.transform(X)
      X = pd.concat([X['engagementMetricsDate'], encoded_X, SelectNumerical().fit_transform(X)], axis=1)
    
    return X


In [84]:
# make Traing
# df_train = df[(df.date >= 20200401) & (df.date <= 20210331)]
df_train = df[df.date <= 20210331]
dts = TrainDataset(df_train, df_train.shape[0])

df_dairy, df_submission, targets = next(dts)

pre = PreProcessor()

X = pre.pre_process(df_dairy, df_submission)

# join targets
df_targets = pd.concat(map(lambda z: unpack_json(z), targets), axis=0)
df_targets['engagementMetricsDate'] = pd.to_datetime(df_targets['engagementMetricsDate'])
y = pd.merge(X[['engagementMetricsDate', 'playerId']], df_targets, on=['engagementMetricsDate', 'playerId'], how='left')
del df_targets



In [61]:
# make Validation
df_valid = df[(df.date >= 20210401) & (df.date <= 20210430)]
dts = TrainDataset(df_valid, df_valid.shape[0])

df_dairy, df_submission, targets = next(dts)

X_val = pre.pre_process(df_dairy, df_submission, phase='eval')
df_targets = pd.concat(map(lambda z: unpack_json(z), targets), axis=0)
df_targets['engagementMetricsDate'] = pd.to_datetime(df_targets['engagementMetricsDate'])
y_val = pd.merge(X_val[['engagementMetricsDate', 'playerId']], df_targets, on=['engagementMetricsDate', 'playerId'], how='left')

del df_targets

In [50]:
gc.collect()

236

In [ ]:
import lightgbm as lgb

params = {'objective': 'regression',
             'metric': 'rmse',
             'verbose': -1,
             'feature_pre_filter': False,
             'lambda_l1': 1.9246603611247695,
             'lambda_l2': 0.0015207873611208637,
             'num_leaves': 45,
             'feature_fraction': 0.616,
             'bagging_fraction': 1.0,
             'bagging_freq': 0,
             'min_child_samples': 20,
}

target_columns = ['target1', 'target2', 'target3', 'target4']

models = []
for target_column in target_columns:
  lgb_train = lgb.Dataset(X.drop(['engagementMetricsDate'], axis=1), y[target_column])
  lgb_valid = lgb.Dataset(X_val.drop(['engagementMetricsDate'], axis=1), y_val[target_column])
  model = lgb.train(params, lgb_train, num_boost_round=200, valid_sets=lgb_valid, early_stopping_rounds=10)
  models.append(model)

In [121]:
df_valid = df[(df.date >= 20210402) & (df.date <= 20210430)]
iter_test = TrainDataset(df_valid, 5)

df_dairy, df_submission, _ = next(iter_test)

In [124]:
X = df_submission.drop(['target1', 'target2', 'target3', 'target4'], axis=1)
X['date'] = pd.to_datetime(X['date'].astype(str))
X['year'] = X['date'].dt.year
X['month'] = X['date'].dt.month
X['day'] = X['date'].dt.day
X['dayofweek'] = X['date'].dt.dayofweek
X['engagementMetricsDate'] = pd.to_datetime(X['date_playerId'].str.split('_', expand = True)[0])
X['playerId'] = X['date_playerId'].str.split('_', expand = True)[1].astype(int)

# join rosters
df_rosters = pd.concat(map(lambda x: unpack_json(x), df_dairy[df_dairy['rosters'].notnull()]['rosters']), axis=0)
df_rosters['gameDate'] = pd.to_datetime(df_rosters['gameDate'])
X = pd.merge(X, df_rosters, how='left', left_on=['date', 'playerId'], right_on=['gameDate', 'playerId']).drop(['gameDate','status'], axis=1)
del df_rosters

# join player twitter followers
df_playerTwitterFollowers = pd.concat(map(lambda x: unpack_json(x),df_dairy[df_dairy['playerTwitterFollowers'].notnull()]['playerTwitterFollowers']),axis=0)
df_playerTwitterFollowers = df_playerTwitterFollowers[['date', 'playerId', 'numberOfFollowers']]
df_playerTwitterFollowers['year'] = df_playerTwitterFollowers['date'].dt.year
df_playerTwitterFollowers['month'] = df_playerTwitterFollowers['date'].dt.month
df_playerTwitterFollowers = df_playerTwitterFollowers.drop('date', axis=1)
X = pd.merge(X, df_playerTwitterFollowers, how='left', on=['playerId', 'year', 'month'])
del df_playerTwitterFollowers

# join team twitter followers
df_teamTwitterFollowers = pd.concat(map(lambda x: unpack_json(x),df_dairy[df_dairy['teamTwitterFollowers'].notnull()]['teamTwitterFollowers']),axis=0)
df_teamTwitterFollowers = df_teamTwitterFollowers[['date', 'teamId', 'numberOfFollowers']]
df_teamTwitterFollowers['year'] = df_teamTwitterFollowers['date'].dt.year
df_teamTwitterFollowers['month'] = df_teamTwitterFollowers['date'].dt.month
df_teamTwitterFollowers = df_teamTwitterFollowers.drop('date', axis=1)
X = pd.merge(X, df_teamTwitterFollowers, how='left', on=['teamId', 'year', 'month'])
del df_teamTwitterFollowers

ValueError: ignored

In [129]:
map(lambda x: unpack_json(x),df_dairy[df_dairy['playerTwitterFollowers'].notnull()][0])

KeyError: ignored

In [131]:
df_dairy[df_dairy['playerTwitterFollowers'].notnull()]

,date,games,rosters,playerBoxScores,teamBoxScores,transactions,standings,awards,events,playerTwitterFollowers,teamTwitterFollowers


In [ ]:
X

In [100]:
if 'google.colab' in sys.modules:
  for df_dairy, df_submission, _ in iter_test:
    print(df_dairy.shape)
    print(df_submission.shape)
    X = pre.pre_process(df_dairy, df_submission, phase='eval')
    X = X.drop('engagementMetricsDate', axis=1)
    # for (i, model) in enumerate(models):
    #   df_submission[target_columns[i]] = model.predict(X)

(5, 11)
(5935, 6)
(5, 11)
(5935, 6)


ValueError: ignored

In [96]:
df_submission

,date,date_playerId,target1,target2,target3,target4
0,20210406,20210407_656669,0.0,0.0,0.0,0.0
1,20210406,20210407_543475,0.0,0.0,0.0,0.0
2,20210406,20210407_592866,0.0,0.0,0.0,0.0
3,20210406,20210407_452678,0.0,0.0,0.0,0.0
4,20210406,20210407_570257,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...
1182,20210410,20210411_593590,0.0,0.0,0.0,0.0
1183,20210410,20210411_642180,0.0,0.0,0.0,0.0
1184,20210410,20210411_663399,0.0,0.0,0.0,0.0
1185,20210410,20210411_664199,0.0,0.0,0.0,0.0



PYDEV DEBUGGER WARNING:
sys.settrace() should not be used when the debugger is being used.
This may cause the debugger to stop working correctly.
If this is needed, please check: 
http://pydev.blogspot.com/2007/06/why-cant-pydev-debugger-work-with.html
to see how to restore the debug tracing back correctly.
Call Location:
  File "/usr/lib/python3.7/bdb.py", line 332, in set_trace
    sys.settrace(self.trace_dispatch)



--Return--
None
> <ipython-input-94-d84b35828fcc>(1)<module>()
----> 1 from IPython.core.debugger import Pdb; Pdb().set_trace()
      2 pre.pre_process(df_dairy, df_submission, phase='eval')

ipdb> d
*** Newest frame
ipdb> d
*** Newest frame
ipdb> d
*** Newest frame
ipdb> d
*** Newest frame
ipdb> d
*** Newest frame
ipdb> d
*** Newest frame
ipdb> q



PYDEV DEBUGGER WARNING:
sys.settrace() should not be used when the debugger is being used.
This may cause the debugger to stop working correctly.
If this is needed, please check: 
http://pydev.blogspot.com/2007/06/why-cant-pydev-debugger-work-with.html
to see how to restore the debug tracing back correctly.
Call Location:
  File "/usr/lib/python3.7/bdb.py", line 357, in set_quit
    sys.settrace(None)



BdbQuit: ignored

In [ ]:
if 'kaggle_secrets' in sys.modules:  # only run while on Kaggle
    import mlb

    env = mlb.make_env()
    iter_test = env.iter_test()

    for (test_df, sample_prediction_df) in iter_test:
    
        # Example: unpack a dataframe from a json column
        today_games = unpack_json(test_df['games'].iloc[0])
    
        # Make your predictions for the next day's engagement
        sample_prediction_df['target1'] = 100.00
    
        # Submit your predictions 
        env.predict(sample_prediction_df)